In [1]:
#  --------------------------库导入----------------------------
import xlwings as xw
import pandas as pd
import difflib
import re
import os 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# # ----------------------------------- 文件预处理 ------------------------------------ 
# def prepocess(path,app):
#     wb = app.books.open(path)
#     #表格列名固定
    
#     #表格横向索引小计栏固定

    


In [ ]:
# ------------------------------------提取详细数据------------------------------------ 
def total_a(path,name,app,j):
    wb = app.books.open(path)
    # #表格总数
    sht_ai =  len(wb.sheets)

    #sht_aname 取出所有表名
    t = [l for l in wb.sheets]
    sht_aname = list()
    for i in range(0,sht_ai):
        sht_aname.append(t[i].name)
    #sht_aname_index 取出周工作表序号的bool list
    sht_aname_index = list()
    for i in sht_aname:
        if "周工作" in i: sht_aname_index.append(True)
        else:sht_aname_index.append(False)


    t = [i for i in range(0,sht_ai)]
    sht_real_ai = np.array(t)[np.array(sht_aname_index)]


    #当前表格
    cur = 1
    # #输出excel的保存位置
    # path_save = './output/22M10/汇总明细.xlsx'                                   #CH

    for sht_i in sht_real_ai:
    
        sht = wb.sheets[sht_i]
        #取表格使用范围
        # sht_value = sht.range('A1:T30').value
        sht_value = sht.range((1,1),sht.used_range.shape).value
        #取得包含任务序号的第一列
        first_col_list  = [i[0] for i in sht_value]
        frnum = first_col_list.index('任务编号') + 1
        #正则找到第一列小计元素
        first_col_list = [str(v) for v in first_col_list]
        resr = difflib.get_close_matches('小计', first_col_list, 1, cutoff=0.5)
        #索引该元素
        lrnum = first_col_list.index(resr[0]) 

        # num_list_right =  list(filter(None, first_col_list[frnum:lrnum])) 
        # wnum = len(num_list_right) #任务数
        rnum = lrnum - frnum #行数

        #表头行的None值转化为字符形式
        first_row_list  = sht_value[1]
        first_row_list = ['None' if v is None else v for v in first_row_list]
        #正则找到项目用时统计的元素位置
        resc = difflib.get_close_matches('项目用时统计', first_row_list, 1, cutoff=0.5)
        #索引该元素得到
        cnum = sht_value[1].index(resc[0]) #列数

        #数据框提取
        worktemp = pd.DataFrame(sht_value[frnum-1:lrnum+1])
        worktemp = worktemp.iloc[:,0:(cnum+1)]
        rowname = worktemp.iloc[0]
        worktemp = worktemp.rename(columns=rowname).iloc[1:]
        worktemp.columns = [*worktemp.columns[:-1], '项目投入统计']
        worktemp = worktemp.reset_index(drop=True)
    
        
        sum_st_col = worktemp.columns.get_loc('星期一')
        sum_sp_col = cnum 
        daynum = sum_sp_col - sum_st_col

        # # 数据矫正
        worktemp['项目投入统计'] = worktemp.iloc[:,sum_st_col:sum_sp_col].sum(axis=1)
        if (worktemp['项目投入统计'][0:-1].sum()) == (worktemp['项目投入统计'].iloc[-1]):
            # print('项目投入统计正确')
            pass
        # #工作天未自增1h
        # elif ((worktemp['项目投入统计'].iloc[-1]) - worktemp['项目投入统计'][0:-1].sum()) == daynum:
        #     # worktemp['项目投入统计'].iloc[:-1] = worktemp['项目投入统计'].iloc[:-1] + (daynum/rnum)
        #     # worktemp['项目投入统计'].iloc[-1] = worktemp['项目投入统计'].iloc[-1] + daynum
        #     print(name[0]+'项目投入统计数值在第'+ str(sht_i) +'周出错（加和原因）')
        else:
            print(name[0]+'项目投入统计数值在第'+ str(sht_i) +'周出错（填报原因）')



        if os.path.isfile(path_save):

            wb0 = xw.books.open (path_save)
            wb0.sheets[j].name = name[0]
            wb0.sheets[j].range(cur,1).expand().value = worktemp.values.tolist()
            wb0.save()

        else:
            wb0 = xw.Book()
            wb0.sheets[0].name = name[0]
            wb0.sheets[0].range(cur,1).expand().value = worktemp.values.tolist()
            wb0.save(path_save)
            wb0.close()



        cur = cur + rnum + 1  
    app.quit()


In [ ]:
# ------------------------------------汇总明细数据------------------------------------ 
def total_b(path,name,app):
    wb = app.books.open(path)
    # #表格总数
    sht_ai =  len(wb.sheets)

    #sht_aname 取出所有表名
    t = [l for l in wb.sheets]
    sht_aname = list()
    for i in range(0,sht_ai):
        sht_aname.append(t[i].name)
    #sht_aname_index 取出周工作表序号的bool list
    sht_aname_index = list()
    for i in sht_aname:
        if "周工作" in i: sht_aname_index.append(True)
        else:sht_aname_index.append(False)


    t = [i for i in range(0,sht_ai)]
    sht_real_ai = np.array(t)[np.array(sht_aname_index)]

    #docker作为回传数据容器
    datedocker = list()
    # rowdocker = list()


    for sht_i in sht_real_ai:
    
        sht = wb.sheets[sht_i]
        #取表格使用范围
        # sht_value = sht.range('A1:T30').value
        sht_value = sht.range((1,1),sht.used_range.shape).value
        #取得包含任务序号的第一列
        first_col_list  = [i[0] for i in sht_value]
        frnum = first_col_list.index('任务编号') + 1
        #正则找到第一列小计元素
        first_col_list = [str(v) for v in first_col_list]
        resr = difflib.get_close_matches('小计', first_col_list, 1, cutoff=0.5)
        #索引该元素
        lrnum = first_col_list.index(resr[0]) 

        # num_list_right =  list(filter(None, first_col_list[frnum:lrnum])) 
        # wnum = len(num_list_right) #任务数
        rnum = lrnum - frnum #行数

        #表头行的None值转化为字符形式
        first_row_list  = sht_value[1]
        first_row_list = ['None' if v is None else v for v in first_row_list]
        #正则找到项目用时统计的元素位置
        resc = difflib.get_close_matches('项目用时统计', first_row_list, 1, cutoff=0.5)
        #索引该元素得到
        cnum = sht_value[1].index(resc[0]) #列数

        #数据框提取
        worktemp = pd.DataFrame(sht_value[frnum-1:lrnum+1])
        worktemp = worktemp.iloc[:,0:(cnum+1)]
        rowname = worktemp.iloc[0]
        worktemp = worktemp.rename(columns=rowname).iloc[1:]
        worktemp.columns = [*worktemp.columns[:-1], '项目投入统计']
        worktemp = worktemp.reset_index(drop=True)
    
        
        sum_st_col = worktemp.columns.get_loc('星期一')
        sum_sp_col = cnum 
        # daynum = sum_sp_col - sum_st_col

        # # 数据矫正
        worktemp['项目投入统计'] = worktemp.iloc[:,sum_st_col:sum_sp_col].sum(axis=1)
        if (worktemp['项目投入统计'][0:-1].sum()) == (worktemp['项目投入统计'].iloc[-1]):
            # print('项目投入统计正确')
            pass
        # #工作天未自增1h
        # elif ((worktemp['项目投入统计'].iloc[-1]) - worktemp['项目投入统计'][0:-1].sum()) == daynum:
        #     # worktemp['项目投入统计'].iloc[:-1] = worktemp['项目投入统计'].iloc[:-1] + (daynum/rnum)
        #     # worktemp['项目投入统计'].iloc[-1] = worktemp['项目投入统计'].iloc[-1] + daynum
        #     print(name[0]+'项目投入统计数值在第'+ str(sht_i) +'周出错（加和原因）')
        else:
            print(name[0]+'项目投入统计数值在第'+ str(sht_i) +'周出错（填报原因）')


        # # 取得数据
        #清洗数据框
        worktemp_final = worktemp.iloc[:-1,:] #删除 小计 行
        worktemp_final = worktemp_final.loc[:,['任务分类','项目名称']] #取出相应列
        worktemp_final['项目用时统计'] = worktemp.iloc[:,-1] #统计用时统一名称

        if ((worktemp_final.iloc[:,0:2].isnull().sum().sum())!= 0):                     #判断前两列是否存在空值                                 |仍有逻辑漏洞
                                                                                        #    因为下面删除0行后空白列取上面的值可能出现逻辑错误     | 当前两列首航为空值
                                                                                        # -----------------------------列空白处理在这里完成     | 第一个值取值溢出
            #去除列的空值
            
            # judge_l = list()                          #每列空值所在行数
            # for z in range(0,2):                      #按列循环
            #     for i in range(0,len(worktemp_final.iloc[:,z])):    #控制所在行数确认  error
            #         if (pd.isnull(worktemp_final.iloc[:,z][i])):
            #             judge_l.append(i)
            #     for j in judge_l:                      #每列空值循环向上取值
            #         worktemp_final.iloc[j,z] = worktemp_final.iloc[j-1,z]

            for j in range(0,2):                                    #列
                for i in range(0,len(worktemp_final.iloc[:,j])):    #行
                    if(pd.isnull(worktemp_final.iloc[:,j][i])):
                        worktemp_final.iloc[i,j] = worktemp_final.iloc[i-1,j]
            
        t = worktemp_final.iloc[:,-1]                                                  ##
        t = t.map(lambda x: (np.isnan(x)) or (x is None) or (x == " ") or (x == 0))     ##去除统计用时为0的工作任务
        worktemp_final = worktemp_final[~(t)]                                          ##


        #当某周为空白时,删除改dataframe
        if (worktemp_final.shape[0] != 0):
            #回传真实数据
            datedocker.append(worktemp_final) 
            # #回传真实行数
            # rowdocker.append(worktemp_final.shape[0])

    # app.quit()  wb.close()  datewiritten函数还需要使用wx.App 所以在这不需要退出
    
    return datedocker

In [ ]:
# ------------------------------------ 数据页签轮询插入 ------------------------------------ 
# def datawritten(path_save,name,app,j,datadocker,rowdocker):
def datawritten_a(path_save,name,app,j,datadocker):   
    # # 接收端数据清洗
    datadockertemp = list()
    df = pd.DataFrame()
    # 暂时合并任务相关的两列 并 将任务作为列名方便concat合并当月所有任务用时(最后还需转置回来)
    for i in datadocker:
        temp = pd.DataFrame()
        temp["任务"] = i['任务分类'] + '_' + i['项目名称']
        temp['投入用时'] = i['项目用时统计']
        temp = temp.groupby(['任务']).sum()         #                         -------------------------------合并处理在这里完成
        temp = temp.T
        # temp = temp.reset_index(drop=True)
        datadockertemp.append(temp)
    
    df = pd.concat(datadockertemp) #员工当月数据以任务为列名合并（省去任务不同需要增加行的代码逻辑，任务不同自增一列）
    df = df.T                      #转置回正常统计维度 (周为列)                     |项目投入(第一周)|项目投入(第二周)|...| ... |
    df = df.reset_index()          #                                              |任务|项目投入(第一周)|项目投入(第二周)|...| ... |
    df['员工'] = name[0]           #                                              |任务|项目投入(第一周)|项目投入(第二周)|...| ... |员工|
    df = pd.concat([df, df['任务'].str.split('_', expand=True)], axis=1)#         |任务|项目投入(第一周)|项目投入(第二周)|...| ... |员工|0|1|   (0，1分别为任务分类和任务名称)
    df_final = pd.concat([df.iloc[:,-3:],df.iloc[:,1:-3]],axis=1)       #         重新对列进行排序
    if os.path.isfile(path_save):

        wb0 = xw.books.open (path_save)
        wb0.sheets[j].name = name[0]
        wb0.sheets[j].range(1,1).expand().value =  df_final.values.tolist()
        wb0.save() 
        wb0.close()

    else:
        wb0 = xw.Book()
        wb0.sheets[0].name = name[0]
        wb0.sheets[0].range(1,1).expand().value = df_final.values.tolist()
        wb0.save(path_save)
        wb0.close()

    # app.kill()

In [ ]:
# ------------------------------------ 数据单页连续插入 ------------------------------------ 
# def datawritten(path_save,name,app,j,datadocker,rowdocker):
def datawritten_b(path_save,name,app,j,datadocker,cur):   
    # # 接收端数据清洗
    datadockertemp = list()
    df = pd.DataFrame()
    # 暂时合并任务相关的两列 并 将任务作为列名方便concat合并当月所有任务用时(最后还需转置回来)
    for i in datadocker:
        temp = pd.DataFrame()
        temp["任务"] = i['任务分类'] + '_' + i['项目名称']
        temp['投入用时'] = i['项目用时统计']
        temp = temp.groupby(['任务']).sum()         #                         -------------------------------合并处理在这里完成
        temp = temp.T
        # temp = temp.reset_index(drop=True)
        datadockertemp.append(temp)
    
    df = pd.concat(datadockertemp) #员工当月数据以任务为列名合并（省去任务不同需要增加行的代码逻辑，任务不同自增一列）
    df = df.T                      #转置回正常统计维度 (周为列)                     |项目投入(第一周)|项目投入(第二周)|...| ... |
    df = df.reset_index()          #                                              |任务|项目投入(第一周)|项目投入(第二周)|...| ... |
    df['员工'] = name[0]           #                                              |任务|项目投入(第一周)|项目投入(第二周)|...| ... |员工|
    df = pd.concat([df, df['任务'].str.split('_', expand=True)], axis=1)#         |任务|项目投入(第一周)|项目投入(第二周)|...| ... |员工|0|1|   (0，1分别为任务分类和任务名称)
    df_final = pd.concat([df.iloc[:,-3:],df.iloc[:,1:-3]],axis=1)       #         重新对列进行排序
    
    if os.path.isfile(path_save):

        wb0 = xw.books.open (path_save)
        wb0.sheets[0].range(cur,1).expand().value =  df_final.values.tolist()
        wb0.save() 
        wb0.close()

    else:
        wb0 = xw.Book()
        wb0.sheets[0].name = "项目投入统计(人员维度)"
        wb0.sheets[0].range(1,1).expand().value = df_final.values.tolist()
        wb0.save(path_save)
        wb0.close()
        
    cur = cur + df_final.shape[0]
    app.kill()
    return cur

In [ ]:
# ------------------------------------ 主程序入口 ------------------------------------ 
def main():
    #------------全局变量-------------------------------------------------
    #将文件夹中所有excel路径存入list
    p = os.getcwd()
    fl = os.listdir('./input/itest')                                  #CH
    l = [i for i in fl if '工作周报' in i]                            #CH
    # l.remove('汇总.xlsx')
    l = [p+"\\input\\itest\\"+i for i in l]                           #CH
    #输出excel的保存位置
    path_save = './output/otest/汇总.xlsx'                             #CH
    #
    j = 0 #汇总明细excel页签flag

    cur = 1 #当页插入行数
    #---------------------------------------------------------------------

    #循环从excel指定周工作表中提取明细数据
    for g in l:
        path = g
        name = re.findall('周报_(.*)_',path)                        #CH
        app = xw.App(visible=True,add_book=False)
        # app.display_alerts=False
        # app.screen_updating=False
        # prepocess(path,app)
        # total_a(path,name,app,j)
        datadocker = total_b(path,name,app)
        # datawritten_a(path_save,name,app,j,datadocker)
        j += 1
        cur = datawritten_b(path_save,name,app,j,datadocker,cur)
        

In [18]:
# ------------------------------------主程序调用------------------------------------ 
main() 